# Dependences

In [13]:
include("../../src/struct_data.jl")

save_data

# Model

In [14]:
@time model = ModelSet(
    TimeModel(
        tₛᵢₘ  = 150000.0,
        dt    = 0.5,
        nₖₙₙ  = 100,
        nₛₐᵥₑ = 50
    ),
    InputModel(
        outer_ratio = 0.8,
        path_input  = "../../data/init/Sphere"
    ),
    OutputModel(
        name_output = "Test_1",
        path_output = ""
    ) 
)

dump(model)

  0.000002 seconds (4 allocations: 144 bytes)
ModelSet
  Time: TimeModel
    tₛᵢₘ: Float64 150000.0
    dt: Float64 0.5
    nₖₙₙ: Int64 100
    nₛₐᵥₑ: Int64 50
  Input: InputModel
    outer_ratio: Float64 0.8
    path_input: String "../../data/init/Sphere"
  Output: OutputModel
    name_output: String "Test_1"
    path_output: String ""


# Aggregate

In [15]:
agg = nothing
# @time @start_agg FusionAGG = FusionAggregate(
@time agg = FusionAggregate(
    [
        AggType(
            "HEK_1", 
            InteractionPar(
                Cubic(0.01,2.0,3.0), 
                ContractilePar(0.001)
            ),
            Float64.(readdlm("../../data/init/Sphere/15.0.xyz")[3:end,2:end]) |> cu
        ),
        AggType(
            "HEK_2", 
            InteractionPar(
                Cubic(0.01,2.0,3.0), 
                ContractilePar(0.001)
            ),
            Float64.(readdlm("../../data/init/Sphere/15.0.xyz")[3:end,2:end]) |> cu
        )
    ], 
    model
)
# show_aggregates(agg)

LoadError: CUDA error: an illegal instruction was encountered (code 715, ERROR_ILLEGAL_INSTRUCTION)

# kNN

In [ ]:
include("../../src/neighbor.jl")
println("------------------------ SIZE ------------------------")
threads=(100)
@cuda(
    threads=threads,
    blocks=cld.(size(agg.Position,1),threads),
    shmem=prod(threads.+2)*sizeof(Float32),
    dist_kernel!(
        agg.Simulation.Neighbor.idx_red,
        agg.Simulation.Neighbor.idx_cont,
        agg.Simulation.Neighbor.idx_sum,
        agg.Simulation.Neighbor.dist,
        agg.Position,
        agg.Simulation.Parameter.Force.rₘₐₓ
    )
)   

println("------------------------ IDX -------------------------")
display(agg.Simulation.Neighbor.idx_red)

# Force

## Function

In [ ]:
include("../../src/forces/forces.jl")

threads=(64,3)
@cuda(
    threads = threads,
    blocks = cld.(size(agg.Position,),threads),
    shmem=prod(threads.+6)*sizeof(Float32),
    sum_force!(
        agg.Position,
        agg.Simulation.Force.F,
        agg.Simulation.Force.Pol,
        agg.Simulation.Neighbor.idx_sum,
        agg.Simulation.Neighbor.idx_red,
        agg.Simulation.Parameter.Force,
        agg.Simulation.Parameter.Contractile.fₚ,
        atan(1),
        model.Time.dt
    )
)
display(agg.Simulation.Force.F')
# display(agg.Simulation.Force.Pol')